In [1]:

import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import weightedstats as ws


# consider using this for weighted stats: http://www.ccgalberta.com/pygeostat/welcome.html



In [2]:
def weighted_mean(var, wts):
    """Calculates the weighted mean"""
    return np.average(var, weights=wts)

def weighted_median_series(val, weight):
    """Calculates the weighted median
    ArithmeticError
    If the sum of the weights is zero, or if the weights are not positive.
    """
    try:
        df = pd.DataFrame({"val": val, "weight": weight})
        df_sorted = df.sort_values("val")
        cumsum = df_sorted["weight"].cumsum()
        cutoff = df_sorted["weight"].sum() / 2.
        result = df_sorted[cumsum >= cutoff]["val"].iloc[0]
        # return just the value
    except:
        result = np.nan
    return result


def weighted_median(df, val_column, weight_column):
    """Calculates the weighted median
    ArithmeticError
    If the sum of the weights is zero, or if the weights are not positive.
    """
    df_sorted = df.sort_values(val_column)
    cumsum = df_sorted[weight_column].cumsum()
    cutoff = df_sorted[weight_column].sum() / 2.
    return df_sorted[cumsum >= cutoff][val_column].iloc[0]

def weighted_variance(var, wts):
    """Calculates the weighted variance"""
    return np.average((var - weighted_mean(var, wts))**2, weights=wts)


def weighted_skew(var, wts):
    """Calculates the weighted skewness"""
    return (np.average((var - weighted_mean(var, wts))**3, weights=wts) /
            weighted_variance(var, wts)**(1.5))

def weighted_kurtosis(var, wts):
    """Calculates the weighted skewness"""
    return (np.average((var - weighted_mean(var, wts))**4, weights=wts) /
            weighted_variance(var, wts)**(2))



def run_weighted_median_on_grouped_df(df, groupby_column, value_column, weight_column):
    """Calculate the weighted median of a dataframe grouped by a column.
    Args:
        df (pandas.DataFrame): DataFrame to calculate weighted median on.
        groupby_column (str): Column to group by.
        value_column (str): Column to calculate weighted median on.
        weight_column (str): Column to use as weight.
    Returns:
        pandas.DataFrame: DataFrame with weighted median for each group.
    """
    # initialize empty list
    d = []
    # loop through each group
    for i in df[groupby_column].unique():
        df_group = df[df[groupby_column] == i]
        # if rows in dataframe are more than 1, calculate weighted median
        if len(df_group) > 1:
            median = weighted_median(df_group, value_column, weight_column)
        else:
            median = df_group[value_column].values[0]
        d.append(
            {
                groupby_column: i,
                "median": median,
            }
        )
    return pd.DataFrame(d)


In [3]:

def load_data(data_file: str) -> pd.DataFrame:
    print 
    '''
    Load data from /data directory
    '''
    PATH = pathlib.Path().resolve()
    DATA_PATH = PATH.joinpath("../data").resolve()
    return pd.read_csv(DATA_PATH.joinpath(data_file))



In [4]:


df_zones_input= load_data("GIS/points_5min_15min_dtw_csv.csv")
# import df_input and set index as alpha 3 
df_input = load_data("country_data_with_cycling_distance.csv")

## manyual adjustment of some columns
df_zones_input["dtw_1"] = df_zones_input["dtw_1"] / 1000 # turn to kms
df_zones_input["max distance walking"] = 3.0 # temp, set max distance reachbale for all countriesl, will change to be bespoke soon
df_input["max distance cycling"] = 8 * df_input["trip_velocity_mean"] # temp, set max distance reachbale for all countriesl, will change to be bespoke soon

In [5]:

# this analysis loses some data as the overlap between the rasters is not perfect. To reduce this error, use the 30 arc second data. Too much heavy lifting for my computer to do this at the moment.

#merge df_input and df_zones on ISO_CC. This assings all the country data to each zone.
# join inner will remove some of the data that is not in both datasets


df_zones = df_zones_input.merge(df_input, left_on="ISOCODE", right_on="alpha3", how="inner")


# print(df_zones_input["ISOCODE"].head())
# print(df_input["alpha3"].head())
# print(df_zones["ISOCODE"].head())

# find ABW
print(df_zones[df_zones["ISOCODE"] == "ABW"])



#adjust population to account for 9 values per raster point (2.5 to 5 arc min resoltuions. 9 values per point)
df_zones["AdjPopFloat"] = df_zones["pop_count_15_1"] / 9 

# # convert population density to percent of national population on a per country basis, grouped by ISO_CC
df_zones["pop_density_perc"] = df_zones.groupby("ISOCODE")["AdjPopFloat"].apply(lambda x: x / x.sum())

# multiply population density by population on a per country basis
df_zones["pop_zone"] = df_zones["pop_density_perc"] * df_zones["Population"]

# drop rows where pop_zone is close to zero
# min_pop = 100
# df_zones = df_zones[df_zones["pop_zone"] > min_pop]


             fid       id       left        top      right     bottom  Value  \
1332667  1332781  2268215 -70.083333  12.583333 -70.000000  12.500000    533   
1332668  1332782  2269934 -70.000000  12.583333 -69.916667  12.500000    533   
1332669  1332783  2269935 -70.000000  12.500000 -69.916667  12.416667    533   
1332670  1332784  2271654 -69.916667  12.500000 -69.833333  12.416667    533   
1332671  1332785  2271655 -69.916667  12.416667 -69.833333  12.333333    533   

        ISOCODE  UNSDCODE  NAME0  ...  percent_insufficient_activity  \
1332667     ABW       533  Aruba  ...                      34.308333   
1332668     ABW       533  Aruba  ...                      34.308333   
1332669     ABW       533  Aruba  ...                      34.308333   
1332670     ABW       533  Aruba  ...                      34.308333   
1332671     ABW       533  Aruba  ...                      34.308333   

        Average Weight      power       Crr  trip_velocity_mean  \
1332667      77.214

In [7]:
# sum the population in each zone
df_zones["country_pop_raw"] = df_zones.groupby("ISOCODE")["pop_zone"].transform("sum")
df_zones["country_pop_ratio"] = df_zones.groupby("ISOCODE")["AdjPopFloat"].transform("sum")


## The section below calculates the population per zone that can reach water

Consider putting in household size and bike sharing here

In [9]:
# population with piped water
df_zones["zone_pop_piped"] = df_zones["pop_zone"]*df_zones["Nat Piped"]/100
df_zones["zone_pop_unpiped"] = df_zones["pop_zone"]*df_zones["Nat NonPiped"]/100

# is it possible to reach water with walking/cycling
df_zones["zone_cycling_okay"] = (df_zones["dtw_1"] < df_zones["max distance cycling"])*1 # multiply by 1 to force to binary not true/false
df_zones["zone_walking_okay"] = (df_zones["dtw_1"] < df_zones["max distance walking"])*1

# how many people can collect water in the zone
df_zones["fraction_of_zone_with_cycling_access"] = df_zones["zone_cycling_okay"]* (df_zones["PBO"])/100
df_zones["fraction_of_zone_with_walking_access"] = df_zones["zone_walking_okay"] * 1

#
df_zones["population_piped_with_cycling_access"] = df_zones["fraction_of_zone_with_cycling_access"] * df_zones["zone_pop_piped"]
df_zones["population_piped_with_walking_access"] = df_zones["fraction_of_zone_with_walking_access"] * df_zones["zone_pop_piped"]

# select the maximum between the two
df_zones["population_piped_with_access"] = df_zones[["population_piped_with_cycling_access", "population_piped_with_walking_access"]].max(axis=1)

# zone pop without water
df_zones["zone_pop_with_water"] =  df_zones["population_piped_with_access"] + df_zones["zone_pop_unpiped"]
df_zones["zone_pop_without_water"] = df_zones["pop_zone"] - df_zones["zone_pop_with_water"]




In [10]:
# use groupby to create dataframe of country level data from df_zones
df_countries = df_zones.groupby("ISOCODE").agg({
    "Entity":"first",
    "country_pop_raw": "mean",
    "zone_pop_with_water":"sum",
    "population_piped_with_access":"sum"  ,
    "population_piped_with_cycling_access":"sum",
    "population_piped_with_walking_access":"sum",
    "Nat Piped":"first",
    "region":"first",
    # call the weighted median function on the column
}).reset_index()

# use groupby to create weighted median, needs to be speerate from the above groupby as it uses apply, which can't be used in the same groupby
# needs to use apply because the function required two columns as input
df_median_group = df_zones.groupby(['ISOCODE']).apply(lambda x : pd.Series({'weighted_med':weighted_median(x,"dtw_1","pop_zone")}))

# merge the weighted median back into the df_countries dataframe
df_countries = df_countries.merge(df_median_group, on="ISOCODE")

# drop rows from the dataframe that have Nan in pop_zone and dtw_1
df_zones = df_zones.dropna(subset=["pop_zone", "dtw_1"])

# create summary columns
df_countries["population_without_water"] = df_countries["country_pop_raw"] - df_countries["zone_pop_with_water"]
# create percent
df_countries["percent_without_water"] = df_countries["population_without_water"] / df_countries["country_pop_raw"] * 100
df_countries["percent_with_water"] = df_countries["zone_pop_with_water"] / df_countries["country_pop_raw"] * 100


In [21]:
# Clean up data from spurious country values
# uses libya as the max possible diatnce to water (from Kummu paper results)

# remove any nan rows
df_countries = df_countries.dropna()

# remove any rows where the median is more than 1km more than "LBY" (Libya)'s median
max_distance = df_countries.loc[df_countries["ISOCODE"] == "LBY", "weighted_med"].values[0] + 1
df_countries = df_countries[df_countries["weighted_med"] < max_distance]


In [23]:

# calculate global totals
pop_with_water = df_zones["zone_pop_with_water"].sum()
pop_without_water = df_zones["zone_pop_without_water"].sum()
# and percentage
pop_with_water_perc = pop_with_water / (pop_with_water + pop_without_water)


## fix the below code to work 
print(f"Population with water: {pop_with_water}")
print(f"Population without water: {pop_without_water}")
print(f"Percentage of population with water: {pop_with_water_perc}")





Population with water: 5338125352.282861
Population without water: 2346436942.638678
Percentage of population with water: 0.6946557458205059


In [24]:
# create choropleth map of population with water from df_country

hover_data_list =[
    "Entity",
    "country_pop_raw",
    "zone_pop_with_water",
    "population_piped_with_access",
    "population_piped_with_cycling_access",
    "population_piped_with_walking_access",
    "population_without_water",
    "percent_without_water",
    "percent_with_water",
    "Nat Piped",
    "region",
    "weighted_med"
    ]


choro = px.choropleth(
    title="Percent of Population Has to Relocate",
    data_frame=df_countries,
    locations="ISOCODE",
    height=600,
    color="percent_without_water",
    hover_name="Entity",
    hover_data=hover_data_list
)
choro.layout.coloraxis.colorbar.title = ''
choro.show()


In [25]:
# create bubble chart from df_countries comprising: access to water, piped water, and using population as the size of the bubble
# create a new column for the size of the bubble
df_countries["bubble_size"] = df_countries["country_pop_raw"] / 1000000

# create a new column for the color of the bubble
df_countries["bubble_color"] = df_countries["Nat Piped"]

# create a new column for the text of the bubble
df_countries["bubble_text"] = df_countries["Entity"]

px.scatter(df_countries, x="percent_without_water", y="Nat Piped", size="bubble_size", color="region", hover_name="bubble_text", title="Access to Water vs. Piped Water vs. Population")

Above graph is the money shot. NMeed to fix bubble sizes, maybe add labels

In [26]:
#sort the countries by distance to water, plot the results
df_countries.sort_values(by="weighted_med", inplace=True)
px.line(df_countries, x="Entity", y="weighted_med", color="region", title="Access to Water vs. Distance to Water")
